In [8]:
import pandas as pd
df_univ = pd.read_csv('university_locations.csv')

# 경희대학교의 위도, 경도 정보를 찾을 수 없어 직접 추가
new_university = {
    '대학명': '경희대학교',
    '면적': '407,376㎡',
    '위도': 37.59685,
    '경도': 127.0518
}

# 새 데이터프레임 생성
new_university_df = pd.DataFrame([new_university])

# 데이터프레임에 경희대학교 추가
df_univ = pd.concat([df_univ, new_university_df], ignore_index=True)

print(df_univ)

            대학명            면적         위도          경도
0         서울대학교    4,317,000㎡  37.464827  126.957199
1         연세대학교      952,750㎡  37.566394  126.938707
2         고려대학교      946,177㎡  37.589388  127.032477
3       이화여자대학교      554,927㎡  37.561859  126.946834
4     서울과학기술대학교      504,922㎡  37.631668  127.077481
5         건국대학교  456,392㎡[62]  37.542327  127.075920
6       서울시립대학교      434,004㎡  37.583866  127.058777
7         한양대학교      401,729㎡  37.557176  127.045409
8         삼육대학교      319,537㎡  37.643582  127.106346
9         서강대학교      242,091㎡  37.550944  126.941002
10      서울여자대학교      225,020㎡  37.628113  127.090457
11        국민대학교      191,736㎡  37.610869  126.997289
12      덕성여자대학교      188,260㎡  37.651298  127.016573
13        중앙대학교      179,972㎡  37.504556  126.956938
14        상명대학교      174,763㎡  37.602638  126.955252
15        동국대학교      171,904㎡  37.558288  127.000167
16      성신여자대학교      156,419㎡  37.591300  127.022107
17       성균관대학교      154,104㎡  37.588227  126.

In [9]:
df_univ.rename(columns={'면적': '면적(㎡)'}, inplace=True)

# 면적 값 정리 및 결측값 처리
df_univ['면적(㎡)'] = df_univ['면적(㎡)'].fillna(0).astype(str).str.replace(',', '').str.extract('(\d+)')[0].astype(int)

# 대학명에서 불필요한 문자 제거
df_univ['대학명'] = df_univ['대학명'].str.replace(r'\[.*?\]', '', regex=True).str.strip()

# 반경 계산 함수
def calculate_radius(area):
    # 면적 100,000㎡ 이하 학교는 500m 
    if area <= 100000:
        return 500
    # 면적 1000,000㎡ 이상 학교는 2000m
    elif area > 1000000:
        return 2000
    else:
        # 100,000㎡ 초과 시 600m, 100,000㎡ 당 100m씩 추가
        additional_radius = ((area - 100000) // 100000 + 1) * 100 
        return 500 + additional_radius

# 반경 계산 및 정렬
df_univ['반경(m)'] = df_univ['면적(㎡)'].apply(calculate_radius)
df_univ_sorted = df_univ.sort_values(by='면적(㎡)', ascending=False)

# 결과 출력
print(df_univ_sorted)

          대학명    면적(㎡)         위도          경도  반경(m)
0       서울대학교  4317000  37.464827  126.957199   2000
1       연세대학교   952750  37.566394  126.938707   1400
2       고려대학교   946177  37.589388  127.032477   1400
3     이화여자대학교   554927  37.561859  126.946834   1000
4   서울과학기술대학교   504922  37.631668  127.077481   1000
5       건국대학교   456392  37.542327  127.075920    900
6     서울시립대학교   434004  37.583866  127.058777    900
45      경희대학교   407376  37.596850  127.051800    900
7       한양대학교   401729  37.557176  127.045409    900
8       삼육대학교   319537  37.643582  127.106346    800
9       서강대학교   242091  37.550944  126.941002    700
10    서울여자대학교   225020  37.628113  127.090457    700
11      국민대학교   191736  37.610869  126.997289    600
12    덕성여자대학교   188260  37.651298  127.016573    600
13      중앙대학교   179972  37.504556  126.956938    600
14      상명대학교   174763  37.602638  126.955252    600
15      동국대학교   171904  37.558288  127.000167    600
16    성신여자대학교   156419  37.591300  127.022107 

In [11]:
import pandas as pd

# CSV 파일 읽기 (UTF-8 인코딩)
df_restaurant = pd.read_csv('소상공인시장진흥공단_상가(상권)정보_서울_202409.csv', encoding='utf-8', low_memory=False)

# 데이터프레임의 컬럼 이름 확인
print(df_restaurant.columns.tolist())

['상가업소번호', '상호명', '지점명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드', '상권업종중분류명', '상권업종소분류코드', '상권업종소분류명', '표준산업분류코드', '표준산업분류명', '시도코드', '시도명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '지번코드', '대지구분코드', '대지구분명', '지번본번지', '지번부번지', '지번주소', '도로명코드', '도로명', '건물본번지', '건물부번지', '건물관리번호', '건물명', '도로명주소', '구우편번호', '신우편번호', '동정보', '층정보', '호정보', '경도', '위도']


In [12]:

# 필요한 열 목록 정의
columns_to_keep = ['상호명', '상권업종대분류코드', '상권업종대분류명', 
                   '상권업종중분류코드', '상권업종중분류명', '상권업종소분류코드', 
                   '상권업종소분류명', '경도', '위도']

# 필요한 column만 남기기
df_restaurant_filtered = df_restaurant[columns_to_keep]

# 결과 확인
print(df_restaurant_filtered.head())

        상호명 상권업종대분류코드 상권업종대분류명 상권업종중분류코드   상권업종중분류명 상권업종소분류코드 상권업종소분류명  \
0     이상한스냅        M1    과학·기술      M113      사진 촬영    M11301    사진촬영업   
1       토끼정        I2       음식      I201         한식    I20101   백반/한정식   
2       가배차        I2       음식      I201         한식    I20101   백반/한정식   
3  비에이치씨항동점        I2       음식      I210      기타 간이    I21006       치킨   
4   무지개삼성약국        G2       소매      G215  의약·화장품 소매    G21501       약국   

           경도         위도  
0  126.828832  37.542117  
1  126.833022  37.560053  
2  127.116136  37.507170  
3  126.820686  37.481640  
4  127.081499  37.489893  


In [13]:
# 상권업종대분류 코드 'I2'는 음식점업
df_restaurant_final = df_restaurant_filtered[df_restaurant_filtered['상권업종대분류코드'] == 'I2']

# 결과 확인
print(df_restaurant_final.head())

        상호명 상권업종대분류코드 상권업종대분류명 상권업종중분류코드 상권업종중분류명 상권업종소분류코드  상권업종소분류명  \
1       토끼정        I2       음식      I201       한식    I20101    백반/한정식   
2       가배차        I2       음식      I201       한식    I20101    백반/한정식   
3  비에이치씨항동점        I2       음식      I210    기타 간이    I21006        치킨   
7       카페숲        I2       음식      I212    비알코올     I21201        카페   
9    영자's곱창        I2       음식      I201       한식    I20109  곱창 전골/구이   

           경도         위도  
1  126.833022  37.560053  
2  127.116136  37.507170  
3  126.820686  37.481640  
7  126.909939  37.446689  
9  127.082832  37.568503  


In [22]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.Collecting geopy
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   ------------ -------------------------- 41.0/125.4 kB 667.8 kB/s eta 0:00:01
   ------------------------- ------------- 81.9/125.4 kB 770.8 kB/s eta 0:00:01
   ---------------------------------------- 125.4/125.4 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB 2.0 MB/s eta 0:00:00




[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import pandas as pd
import numpy as np
from geopy.distance import great_circle

# 음식점 위치를 NumPy 배열로 변환
restaurant_locations = df_restaurant_final[['위도', '경도']].to_numpy()

# 대학교 위치 및 반경을 NumPy 배열로 변환
university_locations = df_univ_sorted[['위도', '경도']].to_numpy()
university_radii = df_univ_sorted['반경(m)'].to_numpy()
university_names = df_univ_sorted['대학명'].to_numpy()

# 음식점에 대해 반경 내 대학교 찾기
def find_universities_for_restaurant(restaurant_loc):
    universities_within_radius = []
    for uni_loc, radius, uni_name in zip(university_locations, university_radii, university_names):
        distance = great_circle(restaurant_loc, uni_loc).meters  # 거리를 미터로 계산
        if distance <= radius:  # 대학교의 반경 내에 있는지 확인
            universities_within_radius.append(uni_name)
    return universities_within_radius  # 리스트로 반환

# 각 음식점에 대해 대학교 정보를 추가
df_restaurant_final_copy = df_restaurant_final.copy()  # 명시적으로 복사본 생성
df_restaurant_final_copy.loc[:, '대학교'] = [
    ', '.join(find_universities_for_restaurant((row['위도'], row['경도'])))
    for _, row in df_restaurant_final_copy.iterrows()
]

# 대학교가 없는 음식점 행 삭제
df_restaurant_final_copy = df_restaurant_final_copy[df_restaurant_final_copy['대학교'].str.strip() != '']

# 결과 출력
print(df_restaurant_final_copy)

                상호명 상권업종대분류코드 상권업종대분류명 상권업종중분류코드 상권업종중분류명 상권업종소분류코드  상권업종소분류명  \
51          매머드커피외대        I2       음식      I212    비알코올     I21201        카페   
66             썬더치킨        I2       음식      I210    기타 간이    I21006        치킨   
153            멜팅그릴        I2       음식      I210    기타 간이    I21004        버거   
178         제이앤제이슨R        I2       음식      I211       주점    I21104     요리 주점   
279     미스홍대게스트하우스2        I2       음식      I212    비알코올     I21201        카페   
...             ...       ...      ...       ...      ...       ...       ...   
465915           홀짝        I2       음식      I211       주점    I21104     요리 주점   
465947          나이브        I2       음식      I210    기타 간이    I21007  김밥/만두/분식   
465959       육회바른연어        I2       음식      I201       한식    I20108  소고기 구이/찜   
465968        제이엠기획        I2       음식      I201       한식    I20101    백반/한정식   
466017         이문족발        I2       음식      I201       한식    I20103     족발/보쌈   

                경도         

In [15]:
# '연세대학교' 자리에 다른 대학교 이름을 넣어서 확인 가능
matching_rows = df_restaurant_final_copy[df_restaurant_final_copy['대학교'].apply(lambda x: '경희대학교' in x)]

# 결과 출력
print(matching_rows)

              상호명 상권업종대분류코드 상권업종대분류명 상권업종중분류코드 상권업종중분류명 상권업종소분류코드   상권업종소분류명  \
51        매머드커피외대        I2       음식      I212    비알코올     I21201         카페   
1788           깐띤        I2       음식      I205    동남아시아    I20501    베트남식 전문   
3032         꼬기92        I2       음식      I201       한식    I20107  돼지고기 구이/찜   
3586        비스트리트        I2       음식      I201       한식    I20101     백반/한정식   
3773           촨커        I2       음식      I202       중식    I20201        중국집   
...           ...       ...      ...       ...      ...       ...        ...   
464718    우동야마이루식        I2       음식      I203       일식    I20303    일식 면 요리   
465054    공들인기절떡풍        I2       음식      I210    기타 간이    I21007   김밥/만두/분식   
465132    쿠시노아회기점        I2       음식      I211       주점    I21104      요리 주점   
465628  매스플레이트회기점        I2       음식      I211       주점    I21104      요리 주점   
466017       이문족발        I2       음식      I201       한식    I20103      족발/보쌈   

                경도         위도          

In [14]:
# 결과를 CSV 파일로 저장
df_restaurant_final_copy.to_csv('restaurant_with_universities.csv', index=False, encoding='utf-8-sig')